In [11]:
import cv2
import numpy as np
import glob

I = np.array([[1, 0, 0, 0, 0, 0],
              [0, 1, 0, 0, 0, 0],
              [0, 0, 1, 0, 0, 0],
              [0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 1, 0],
              [0, 0, 0, 0, 0, 1]])

def update(x, P, Z, H, R):
    y = Z -np.dot(H, x)
    S = np.dot(np.dot(H, P), np.transpose(H)) + R
    K = np.dot(np.dot(P, np.transpose(H)), np.linalg.pinv(S))
    return [x + np.dot(K, y), np.dot((I - np.dot(K, H)), P)]    
    
def predict(x, P, F, u):
    return [np.dot(F, x) + u, np.dot(np.dot(F, P), np.transpose(F))]
    
### Initialize Kalman filter ###
# The initial state (6x1).
# The initial state. The robot starts in position 0 with the velocity 0.
x = np.array([[0], # Position along the x-axis
              [0], # Velocity along the x-axis
              [0],     #acc on x
              [0], # Position along the y-axis
              [0],# Velocity along the y-axis
              [0]]) #acc on y

# The initial uncertainty (6x6).
P = np.array([[1000, 0, 0, 0, 0, 0],
              [0, 1000, 0, 0, 0, 0],
              [0, 0, 1000, 0, 0, 0],
              [0, 0, 0, 1000, 0, 0],
              [0, 0, 0, 0, 1000, 0],
              [0, 0, 0, 0, 0, 1000]])

# The external motion (6x1).
u = np.array([[0],
              [0],
              [0],
              [0],
              [0],
              [0]])

# The transition matrix (6x6). 
F = np.array([[1, 1, 0.5, 0, 0, 0],
              [0, 1, 1,   0, 0, 0], 
              [0, 0, 1,   0, 0, 0],
              [0, 0, 0,   1, 1, 0.5],
              [0, 0, 0,   0, 1, 1],
              [0, 0, 0,   0, 0, 1]])

# The observation matrix (2x6).
H = np.array([[1, 0, 0, 0, 0, 0],
             [0, 0, 0, 1, 0, 0]])

# The measurement uncertainty.
R = 0.01


    
leftImages = glob.glob('./left/*.png')
previous_frame = cv2.imread(leftImages[0])
#previous_frame = cv2.cvtColor(previous_frame, cv2.COLOR_RGB2GRAY)

backSub = cv2.createBackgroundSubtractorMOG2(500, 16, False)
#backSub = cv2.createBackgroundSubtractorKNN(history=1000, dist2Threshold=1000, detectShadows=False)
#kernel = cv2.getStructuringElement(cv2.MORPH_OPEN, (3, 3), (0, 0))
for img in leftImages[1:]:
    next_frame = cv2.imread(img)
    output = next_frame.copy()
    #next_frame = cv2.cvtColor(next_frame, cv2.COLOR_RGB2GRAY)
    
    
    fgMask = backSub.apply(previous_frame)
    
    kernel = np.ones((5,5), np.uint8) 
    #fgMask = cv2.erode(fgMask, kernel, iterations=1) 
    fgMask = cv2.dilate(fgMask, kernel, iterations=1) 
    
    contours, _ = cv2.findContours(fgMask,2,1)            
    contours = sorted(contours, key=cv2.contourArea)            
    out_mask = np.zeros_like(fgMask)
    cv2.drawContours(out_mask, [contours[-1]], -1, 255, cv2.FILLED, 1)  
    
    res = cv2.bitwise_and(previous_frame,previous_frame,mask = out_mask)
    cv2.imshow('Frame', res)
    cv2.waitKey(50)
    
    previous_frame = next_frame

cv2.destroyAllWindows()